In [2]:
from dotenv import load_dotenv
import boto3
import os
import pymysql

# Database connection parameters
load_dotenv(".env")

endpoint = os.getenv('RDS_ENDPOINT')
username = os.getenv('RDS_USERNAME')
password = os.getenv('RDS_PASSWORD')
database = os.getenv('RDS_DATABASE')
port = 3306


# Establish a connection to the RDS instance
connection = pymysql.connect(
    host=endpoint,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()
# Execute a query
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"Database version: {version}")

Database version: ('8.0.35',)


In [14]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

# Get all table names
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Drop each table
for (table_name,) in tables:
    drop_table_sql = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(drop_table_sql)
    print(f"Dropped table {table_name}")

# Re-enable foreign key checks
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

Dropped table additional_health_tags
Dropped table bridge_table_discounts_to_groceries
Dropped table bridge_table_groceries_to_additional_health_tags
Dropped table bridge_table_products_types_to_recipes
Dropped table discounts
Dropped table groceries
Dropped table groceries_types
Dropped table recipes


0

groceries_types table creation

In [15]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS groceries_types (
    grocery_type_id INT AUTO_INCREMENT PRIMARY KEY,
    grocery_type_name VARCHAR(255) NOT NULL,
    embedding_vector BLOB,
    grocery_category VARCHAR(255) NOT NULL,
    grocery_base_category VARCHAR(255) NOT NULL

);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)


0

recipes table creation

In [16]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS recipes (
    recipe_id INT AUTO_INCREMENT PRIMARY KEY,
    recipe_name VARCHAR(255) NOT NULL,
    instructions_and_ingredients TEXT,
    embedding_vector BLOB,
    recipe_category VARCHAR(255)
    
);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)


0

bridge_table_products_types_to_recipes table creation

In [17]:
create_table_sql = """
CREATE TABLE bridge_table_products_types_to_recipes (
    recipe_id INT,
    grocery_type_id INT,
    PRIMARY KEY (grocery_type_id, recipe_id),
    FOREIGN KEY (recipe_id) REFERENCES recipes(recipe_id),
    FOREIGN KEY (grocery_type_id) REFERENCES groceries_types(grocery_type_id)
);

"""

# Execute the SQL statement
cursor.execute(create_table_sql)

0

products table creation

In [18]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS groceries (
    barcode INT AUTO_INCREMENT PRIMARY KEY,
    grocery_name VARCHAR(255) NOT NULL,
    brand VARCHAR(255) NOT NULL,
    grocery_type_id INT NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    pricing_type ENUM('kg', 'item', 'liter') NOT NULL,
    nutritional_values TEXT, -- Stores nutritional information
    high_level_sugar BOOLEAN DEFAULT false, -- Indicates high sugar content
    high_level_sodium BOOLEAN DEFAULT false, -- Indicates high sodium content
    high_level_saturated_fat BOOLEAN DEFAULT false, -- Indicates high saturated fat content
    sales_count INT DEFAULT 0, -- Added to track the number of sales
    FOREIGN KEY (grocery_type_id) REFERENCES groceries_types(grocery_type_id)
);
"""
 
# Execute the SQL statement
cursor.execute(create_table_sql)


0

In [19]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS additional_health_tags (
    health_tag_id INT AUTO_INCREMENT PRIMARY KEY,
    health_tag_name VARCHAR(255) NOT NULL
);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)

0

In [20]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS bridge_table_groceries_to_additional_health_tags (
    barcode INT NOT NULL,
    health_tag_id INT NOT NULL,
    PRIMARY KEY (barcode, health_tag_id),
    FOREIGN KEY (barcode) REFERENCES groceries(barcode),
    FOREIGN KEY (health_tag_id) REFERENCES additional_health_tags(health_tag_id)
);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)

0

discounts table creation

In [21]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS discounts (
    discount_id INT AUTO_INCREMENT PRIMARY KEY,
    discount_name VARCHAR(255) NOT NULL,
    grocery_type_id INT NOT NULL,
    discount_type ENUM('percentage', 'fixed_price') NOT NULL,
    discount_value DECIMAL(10, 2), 
    grocery_category VARCHAR(255) NOT NULL,
    grocery_base_category VARCHAR(255) NOT NULL,
    FOREIGN KEY (grocery_type_id) REFERENCES groceries_types(grocery_type_id)

);
"""

# Execute the SQL statement
cursor.execute(create_table_sql)


0

bridge_table_discounts_to_groceries table creation

In [22]:
create_table_sql = """
CREATE TABLE bridge_table_discounts_to_groceries (
    barcode INT NOT NULL,
    discount_id INT NOT NULL,
    PRIMARY KEY (barcode, discount_id),
    FOREIGN KEY (barcode) REFERENCES groceries(barcode),
    FOREIGN KEY (discount_id) REFERENCES discounts(discount_id)
);

"""

# Execute the SQL statement
cursor.execute(create_table_sql)

0

In [23]:
# Insert data into groceries_types
insert_groceries_types = """
INSERT INTO groceries_types (grocery_type_name, embedding_vector, grocery_base_category, grocery_category)
VALUES
('חלב 3 אחוז', NULL, 'מוצרי חלב', 'חלב'),
('חלב 1 אחוז', NULL, 'מוצרי חלב', 'חלב'),
('שוקולד חלב', NULL, 'שוקולד', 'שוקולד'),
('שוקולד מריר', NULL, 'שוקולד', 'שוקולד'),
('תפוח אדמה לבן', NULL, 'ירקות', 'תפוח אדמה'),
('תפוח אדמה אדום', NULL, 'ירקות', 'תפוח אדמה'),
('גזר', NULL, 'ירקות', 'גזר'),
('עגבנייה', NULL, 'ירקות', 'עגבנייה'),
('קמח', NULL, 'מוצרי אפייה', 'קמח'),
('מלח', NULL, 'תבלינים', 'מלח'),
('שמן קנולה', NULL, 'שמנים', 'שמן'),
('שמן זית', NULL, 'שמנים', 'שמן'),
('ביצים', NULL, 'ביצים', 'ביצים'),
('יוגורט', NULL, 'מוצרי חלב', 'יוגורט'),
('קפה', NULL, 'שתייה', 'קפה'),
('תה ירוק', NULL, 'שתייה', 'תה'),
('פסטה', NULL, 'מוצרי פסטה', 'פסטה'),
('רוטב עגבניות', NULL, 'רטבים', 'רוטב'),
('מיץ תפוזים', NULL, 'שתייה', 'מיץ'),
('בצל', NULL, 'ירקות', 'בצל'),
('שום', NULL, 'ירקות', 'שום'),
('תפוח עץ ירוק', NULL, 'פירות', 'תפוח עץ'),
('תפוח עץ אדום', NULL, 'פירות', 'תפוח עץ'),
('בננה', NULL, 'פירות', 'בננה'),
('אבוקדו', NULL, 'פירות', 'אבוקדו'),
('לחם לבן', NULL, 'מאפים', 'לחם'),
('לחם מלא', NULL, 'מאפים', 'לחם'),
('חמאה', NULL, 'מוצרי חלב', 'חמאה'),
('גבינת צהובה', NULL, 'מוצרי חלב', 'גבינה'),
('גבינת קוטג', NULL, 'מוצרי חלב', 'גבינה'),
('דבש', NULL, 'ממרחים', 'דבש'),
('שוקולד פררו רושה', NULL, 'ממתקים', 'שוקולד'),
('ציפס', NULL, 'חטיפים', 'ציפס'),
('פריכיות', NULL, 'חטיפים', 'פריכיות'),
('קולה', NULL, 'שתייה', 'משקאות קלים'),
('פיצוחים', NULL, 'חטיפים', 'פיצוחים'),
('חטיף אגוזים', NULL, 'חטיפים', 'חטיף אגוזים'),
('קרמבו', NULL, 'ממתקים', 'ממתק'),
('סודה', NULL, 'שתייה', 'משקאות קלים'),
('חטיף תפו"א', NULL, 'חטיפים', 'חטיף תפו"א'),
('מאפה זיתים', NULL, 'מזון מוכן', 'מאפה'),
('סלט חצילים', NULL, 'מזון מוכן', 'סלטים'),
('רוטב פסטו', NULL, 'רטבים', 'רוטב'),
('תפוח אדמה בייבי', NULL, 'ירקות', 'תפוח אדמה'),
('גזר בייבי', NULL, 'ירקות', 'גזר'),
('עגבנייה שרי', NULL, 'ירקות', 'עגבנייה'),
('עגבנייה תמר', NULL, 'ירקות', 'עגבנייה'),
('קמח תופח', NULL, 'מוצרי אפייה', 'קמח'),
('קמח מלא', NULL, 'מוצרי אפייה', 'קמח'),
('סוכר לבן', NULL, 'מוצרי אפייה', 'סוכר'),
('סוכר חום', NULL, 'מוצרי אפייה', 'סוכר'),
('מלח גס', NULL, 'תבלינים', 'מלח'),
('מלח הימלאיה', NULL, 'תבלינים', 'מלח'),
('קפה נמס', NULL, 'שתייה', 'קפה'),
('קפה פילטר', NULL, 'שתייה', 'קפה'),
('תה שחור', NULL, 'שתייה', 'תה'),
('תה צמחים', NULL, 'שתייה', 'תה'),
('בצל ירוק', NULL, 'ירקות', 'בצל'),
('בצל סגול', NULL, 'ירקות', 'בצל'),
('מיץ ענבים', NULL, 'שתייה', 'מיץ'),
('מיץ רימונים', NULL, 'שתייה', 'מיץ'),
('שום כתוש', NULL, 'ירקות', 'שום'),
('שום שחור', NULL, 'ירקות', 'שום'),
('תפוח עץ זהוב', NULL, 'פירות', 'תפוח עץ'),
('תפוח עץ פינק ליידי', NULL, 'פירות', 'תפוח עץ'),
('בננה בייבי', NULL, 'פירות', 'בננה'),
('מרגרינה', NULL, 'מוצרי אפייה', 'מרגרינה'),
('לחם שיפון', NULL, 'מאפים', 'לחם'),
('לחם דגנים', NULL, 'מאפים', 'לחם'),
('חמאה מלוחה', NULL, 'מוצרי חלב', 'חמאה'),
('גבינת פטה', NULL, 'מוצרי חלב', 'גבינה'),
('גבינת קממבר', NULL, 'מוצרי חלב', 'גבינה'),
('סילאן', NULL, 'ממרחים', 'סילאן'),
('במבה', NULL, 'חטיפים', 'במבה'),
('ביסלי', NULL, 'חטיפים', 'ביסלי'),
('פריכיות תירס', NULL, 'חטיפים', 'פריכיות'),
('פריכיות כוסמין', NULL, 'חטיפים', 'פריכיות'),
('ספרייט', NULL, 'שתייה', 'משקאות קלים'),
('גרעיני חמנייה', NULL, 'חטיפים', 'פיצוחים'),
('פיסטוקים', NULL, 'חטיפים', 'פיצוחים'),
('חטיף גרנולה', NULL, 'חטיפים', 'חטיף גרנולה'),
('חטיף פירות', NULL, 'חטיפים', 'חטיף פירות'),
('מרשמלו', NULL, 'ממתקים', 'מרשמלו'),
('סוכריות גומי', NULL, 'ממתקים', 'סוכריות גומי'),
('מים', NULL, 'שתייה', 'מים'),
('מים מוגזים', NULL, 'שתייה', 'מים מוגזים'),
('מאפה גבינה', NULL, 'מזון מוכן', 'מאפה'),
('בורקס', NULL, 'מזון מוכן', 'בורקס'),
('טחינה', NULL, 'מזון מוכן', 'טחינה'),
('חומוס', NULL, 'מזון מוכן', 'חומוס'),
('רוטב אלפרדו', NULL, 'רטבים', 'רוטב'),
('רוטב סויה', NULL, 'רטבים', 'רוטב'),
('חלב סויה', NULL, 'תחליפי חלב', 'חלב סויה'),
('חלב שקדים', NULL, 'תחליפי חלב', 'חלב שקדים'),
('חלב שיבולת שועל', NULL, 'תחליפי חלב', 'חלב שיבולת שועל'),
('לחם ללא גלוטן', NULL, 'מאפים', 'לחם ללא גלוטן'),
('פסטה ללא גלוטן', NULL, 'מוצרי פסטה', 'פסטה ללא גלוטן'),
('עוגיות ללא גלוטן', NULL, 'ממתקים', 'עוגיות ללא גלוטן'),
('גבינה טבעונית', NULL, 'תחליפי חלב', 'גבינה טבעונית'),
('יוגורט ללא לקטוז', NULL, 'מוצרי חלב', 'יוגורט ללא לקטוז'),
('חמאה טבעונית', NULL, 'תחליפי חלב', 'חמאה טבעונית'),
('שוקולד ללא סוכר', NULL, 'ממתקים', 'שוקולד ללא סוכר'),
('סוכרלוז', NULL, 'ממתיקים', 'תחליפי סוכר'),
('סטיביה', NULL, 'ממתיקים', 'תחליפי סוכר'),
('טחינה גולמית', NULL, 'ממרחים', 'טחינה גולמית'),
('ממרח שוקולד ללא אגוזים', NULL, 'ממרחים', 'ממרח שוקולד ללא אגוזים'),
('חמאת בוטנים', NULL, 'ממרחים', 'חמאת בוטנים'),
('קרקרים ללא גלוטן', NULL, 'חטיפים', 'קרקרים ללא גלוטן'),
('טופו', NULL, 'מזון טבעוני', 'טופו'),
('סייטן', NULL, 'מזון טבעוני', 'סייטן'),
('נקניקיות טבעוניות', NULL, 'מזון טבעוני', 'נקניקיות טבעוניות'),
('המבורגר טבעוני', NULL, 'מזון טבעוני', 'המבורגר טבעוני'),
('יוגורט קוקוס', NULL, 'תחליפי חלב', 'יוגורט קוקוס'),
('גלידה טבעונית', NULL, 'קינוחים', 'גלידה טבעונית'),
('עוגות ללא סוכר', NULL, 'מאפים', 'עוגות ללא סוכר'),
('דגני בוקר ללא גלוטן', NULL, 'דגני בוקר', 'דגני בוקר ללא גלוטן'),
('חלב ללא לקטוז', NULL, 'מוצרי חלב', 'חלב ללא לקטוז'),
('גבינה ללא לקטוז', NULL, 'מוצרי חלב', 'גבינה ללא לקטוז'),
('שמנת ללא לקטוז', NULL, 'מוצרי חלב', 'שמנת ללא לקטוז'),
('ממרח שקדים', NULL, 'ממרחים', 'ממרח שקדים'),
('קמח ללא גלוטן', NULL, 'מוצרי אפייה', 'קמח ללא גלוטן'),
('שוקולד טבעוני', NULL, 'ממתקים', 'שוקולד טבעוני'),
('אטריות אורז', NULL, 'מוצרי פסטה', 'אטריות אורז'),
('קוסקוס ללא גלוטן', NULL, 'מזון מוכן', 'קוסקוס ללא גלוטן'),
('חומוס קפוא', NULL, 'מזון מוכן', 'חומוס קפוא'),
('סלט קישואים', NULL, 'מזון מוכן', 'סלט קישואים'),
('שמן קוקוס', NULL, 'שמנים', 'שמן קוקוס'),
('חטיף חלבון טבעוני', NULL, 'חטיפים', 'חטיף חלבון טבעוני'),
('פירות יבשים', NULL, 'פירות', 'פירות יבשים'),
('גרנולה ללא גלוטן', NULL, 'דגני בוקר', 'גרנולה ללא גלוטן'),
('עוגיות טבעוניות', NULL, 'מאפים', 'עוגיות טבעוניות'),
('חלב קוקוס', NULL, 'תחליפי חלב', 'חלב קוקוס');
"""

cursor.execute(insert_groceries_types)

132

In [24]:
insert_additional_health_tags = """
INSERT INTO additional_health_tags (health_tag_name)
VALUES
('מאושר ע"י האגודה הישראלית לסוכרת'),
('ללא גלוטן'),
('טבעוני'),
('עלול להכיל אגוזים'),
('לא מכיל לקטוז')
"""

cursor.execute(insert_additional_health_tags)


5

In [25]:
insert_groceries = """  
INSERT INTO groceries (grocery_name, brand, grocery_type_id, price, pricing_type, nutritional_values, high_level_sugar, high_level_sodium, high_level_saturated_fat, sales_count)  
VALUES  
('חלב תנובה 3 אחוז', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז'), 6.99, 'liter', 'אנרגיה: 62 קלוריות, חלבון: 3.2 גרם, שומן: 3 גרם, פחמימות: 4.8 גרם', false, false, false, 950),  
('חלב 1 אחוז טרה', 'טרה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 1 אחוז'), 5.99, 'liter', 'אנרגיה: 42 קלוריות, חלבון: 3.4 גרם, שומן: 1 גרם, פחמימות: 4.9 גרם', false, false, false, 750),  
('חלב יטבתה 3 אחוז', 'יטבתה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז'), 7.20, 'liter', 'אנרגיה: 61 קלוריות, חלבון: 3.3 גרם, שומן: 3 גרם, פחמימות: 4.7 גרם', false, false, false, 620),  
('חלב הרדוף 3 אחוז', 'הרדוף', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז'), 7.10, 'liter', 'אנרגיה: 63 קלוריות, חלבון: 3.1 גרם, שומן: 3 גרם, פחמימות: 4.9 גרם', false, false, false, 480),  
('שוקולד חלב עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב'), 8.50, 'item', 'אנרגיה: 534 קלוריות, חלבון: 7 גרם, שומן: 30 גרם, פחמימות: 60 גרם', true, false, true, 1020),  
('שוקולד מריר עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד מריר'), 9.00, 'item', 'אנרגיה: 509 קלוריות, חלבון: 5 גרם, שומן: 37 גרם, פחמימות: 48 גרם', true, false, true, 690),  
('שוקולד חלב מילקה', 'מילקה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב'), 9.50, 'item', 'אנרגיה: 530 קלוריות, חלבון: 6 גרם, שומן: 29 גרם, פחמימות: 61 גרם', true, false, true, 520),  
('שוקולד מריר לינדט 70%', 'לינדט', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד מריר'), 12.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 7 גרם, שומן: 36 גרם, פחמימות: 49 גרם', true, false, true, 450),  
('תפוח אדמה לבן רמי לוי', 'רמי לוי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה לבן'), 3.99, 'kg', 'אנרגיה: 77 קלוריות, חלבון: 2 גרם, שומן: 0.1 גרם, פחמימות: 17 גרם', false, false, false, 880),  
('תפוח אדמה אדום שופרסל', 'שופרסל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה אדום'), 4.20, 'kg', 'אנרגיה: 77 קלוריות, חלבון: 2 גרם, שומן: 0.1 גרם, פחמימות: 17 גרם', false, false, false, 730),  
('תפוח אדמה בייבי', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה בייבי'), 6.00, 'kg', 'אנרגיה: 75 קלוריות, חלבון: 2 גרם, שומן: 0.1 גרם, פחמימות: 17 גרם', false, false, false, 560),  
('גזר אורגני טיב טעם', 'טיב טעם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גזר'), 4.50, 'kg', 'אנרגיה: 41 קלוריות, חלבון: 0.9 גרם, שומן: 0.2 גרם, פחמימות: 10 גרם', false, false, false, 810),  
('גזר טרי חקלאי', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גזר'), 3.80, 'kg', 'אנרגיה: 41 קלוריות, חלבון: 0.9 גרם, שומן: 0.2 גרם, פחמימות: 10 גרם', false, false, false, 960),  
('גזר בייבי ארוז', 'שופרסל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גזר בייבי'), 6.00, 'kg', 'אנרגיה: 35 קלוריות, חלבון: 0.6 גרם, שומן: 0.1 גרם, פחמימות: 8 גרם', false, false, false, 420),  
('עגבנייה', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה'), 7.00, 'kg', 'אנרגיה: 18 קלוריות, חלבון: 0.9 גרם, שומן: 0.2 גרם, פחמימות: 3.9 גרם', false, false, false, 950),  
('עגבנייה שרי', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה שרי'), 9.00, 'kg', 'אנרגיה: 20 קלוריות, חלבון: 1 גרם, שומן: 0.2 גרם, פחמימות: 4 גרם', false, false, false, 680),  
('עגבנייה תמר', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה תמר'), 8.50, 'kg', 'אנרגיה: 19 קלוריות, חלבון: 0.8 גרם, שומן: 0.2 גרם, פחמימות: 4 גרם', false, false, false, 510),  
('קמח שחר', 'שחר', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח'), 2.50, 'kg', 'אנרגיה: 364 קלוריות, חלבון: 10 גרם, שומן: 1 גרם, פחמימות: 76 גרם', true, false, false, 820),  
('קמח תופח אסם', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח תופח'), 4.00, 'kg', 'אנרגיה: 360 קלוריות, חלבון: 9 גרם, שומן: 1 גרם, פחמימות: 75 גרם', true, false, false, 870),  
('קמח מלא ברמן', 'ברמן', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח מלא'), 3.50, 'kg', 'אנרגיה: 340 קלוריות, חלבון: 12 גרם, שומן: 2 גרם, פחמימות: 70 גרם', true, false, false, 430),  
('סוכר לבן נוגה', 'נוגה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכר לבן'), 3.00, 'kg', 'אנרגיה: 387 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 100 גרם', true, false, false, 900),  
('סוכר לבן סוגת', 'סוגת', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכר לבן'), 2.80, 'kg', 'אנרגיה: 387 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 100 גרם', true, false, false, 740),  
('סוכר חום דמררה', 'סוגת', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכר חום'), 4.00, 'kg', 'אנרגיה: 380 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 98 גרם', true, false, false, 540),  
('מלח דק סול', 'סול', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מלח'), 1.50, 'kg', 'אין ערכים תזונתיים למלח', false, true, false, 830),  
('מלח גס', 'מלח הארץ', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מלח גס'), 1.80, 'kg', 'אין ערכים תזונתיים למלח', false, true, false, 970),  
('מלח הימלאיה ורוד', 'הימלאיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מלח הימלאיה'), 5.00, 'kg', 'אין ערכים תזונתיים למלח', false, true, false, 620),  
('שמן זית יד מרדכי', 'יד מרדכי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שמן זית'), 15.00, 'liter', 'אנרגיה: 884 קלוריות, חלבון: 0 גרם, שומן: 100 גרם, פחמימות: 0 גרם', false, false, true, 910),  
('שמן זית כתית מעולה', 'זיתא', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שמן זית'), 18.00, 'liter', 'אנרגיה: 884 קלוריות, חלבון: 0 גרם, שומן: 100 גרם, פחמימות: 0 גרם', false, false, true, 670),  
('שמן קנולה קנוליה', 'קנוליה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שמן קנולה'), 7.50, 'liter', 'אנרגיה: 884 קלוריות, חלבון: 0 גרם, שומן: 100 גרם, פחמימות: 0 גרם', false, false, true, 480),  
('ביצים טריות צבר', 'צבר', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים'), 10.00, 'item', 'אנרגיה: 155 קלוריות, חלבון: 13 גרם, שומן: 11 גרם, פחמימות: 1.1 גרם', false, false, false, 1020),  
('ביצי חופש', 'משק טבעי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים'), 12.00, 'item', 'אנרגיה: 155 קלוריות, חלבון: 13 גרם, שומן: 11 גרם, פחמימות: 1.1 גרם', false, false, false, 740),  
('ביצי אומגה 3', 'משק בריא', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים'), 14.00, 'item', 'אנרגיה: 155 קלוריות, חלבון: 13 גרם, שומן: 11 גרם, פחמימות: 1.1 גרם', false, false, false, 510),  
('יוגורט דנונה', 'דנונה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'יוגורט'), 5.00, 'item', 'אנרגיה: 61 קלוריות, חלבון: 5 גרם, שומן: 3 גרם, פחמימות: 4 גרם', false, false, false, 930),  
('יוגורט פרו ביוטי', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'יוגורט'), 6.00, 'item', 'אנרגיה: 60 קלוריות, חלבון: 5.5 גרם, שומן: 2.5 גרם, פחמימות: 5 גרם', false, false, false, 690),  
('יוגורט יווני 10%', 'מילק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'יוגורט'), 7.00, 'item', 'אנרגיה: 120 קלוריות, חלבון: 5 גרם, שומן: 10 גרם, פחמימות: 3 גרם', false, false, true, 450),  
('קפה טורקי ארקפה', 'ארקפה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קפה'), 12.00, 'item', 'אנרגיה: 2 קלוריות, חלבון: 0.1 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 860),  
('קפה נמס עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קפה נמס'), 10.00, 'item', 'אנרגיה: 3 קלוריות, חלבון: 0.2 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 720),  
('קפה פילטר סטארבקס', 'סטארבקס', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קפה פילטר'), 15.00, 'item', 'אנרגיה: 2 קלוריות, חלבון: 0.1 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 490),  
('תה ירוק עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תה ירוק'), 7.00, 'item', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 820),  
('תה שחור ויסוצקי', 'ויסוצקי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תה שחור'), 6.50, 'item', 'אנרגיה: 1 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0.2 גרם', false, false, false, 970),  
('תה צמחים קמומיל', 'ויסוצקי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תה צמחים'), 8.00, 'item', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 480),  
('פסטה דל', 'דל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה'), 6.00, 'kg', 'אנרגיה: 370 קלוריות, חלבון: 13 גרם, שומן: 1.5 גרם, פחמימות: 75 גרם', true, false, false, 940),  
('פסטה פנה ברילה', 'ברילה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה'), 7.00, 'kg', 'אנרגיה: 360 קלוריות, חלבון: 12 גרם, שומן: 1 גרם, פחמימות: 74 גרם', true, false, false, 710),  
('פסטה ספגטי אסם', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה'), 5.50, 'kg', 'אנרגיה: 370 קלוריות, חלבון: 13 גרם, שומן: 1.5 גרם, פחמימות: 75 גרם', true, false, false, 560),  
('בצל', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בצל'), 2.50, 'kg', 'אנרגיה: 40 קלוריות, חלבון: 1.1 גרם, שומן: 0.1 גרם, פחמימות: 9.3 גרם', false, false, false, 880),  
('בצל ירוק', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בצל ירוק'), 3.00, 'kg', 'אנרגיה: 32 קלוריות, חלבון: 1.8 גרם, שומן: 0.2 גרם, פחמימות: 7 גרם', false, false, false, 720),  
('בצל סגול', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בצל סגול'), 2.80, 'kg', 'אנרגיה: 36 קלוריות, חלבון: 1 גרם, שומן: 0.1 גרם, פחמימות: 8.5 גרם', false, false, false, 510),  
('רוטב עגבניות רוטב אדום', 'רוטב אדום', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב עגבניות'), 4.00, 'item', 'אנרגיה: 29 קלוריות, חלבון: 1.2 גרם, שומן: 0.2 גרם, פחמימות: 6.4 גרם', false, false, false, 940),  
('רוטב עגבניות בזיליקום', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב עגבניות'), 5.00, 'item', 'אנרגיה: 30 קלוריות, חלבון: 1 גרם, שומן: 0.3 גרם, פחמימות: 7 גרם', false, false, false, 680),  
('רוטב עגבניות פיקנטי', 'פריניר', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב עגבניות'), 4.50, 'item', 'אנרגיה: 28 קלוריות, חלבון: 1 גרם, שומן: 0.2 גרם, פחמימות: 6 גרם', false, false, false, 460),  
('מיץ תפוזים פרימור', 'פרימור', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מיץ תפוזים'), 8.00, 'liter', 'אנרגיה: 45 קלוריות, חלבון: 0.7 גרם, שומן: 0.2 גרם, פחמימות: 10.4 גרם', false, false, false, 920),  
('מיץ ענבים טבעי', 'ענביה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מיץ ענבים'), 9.50, 'liter', 'אנרגיה: 60 קלוריות, חלבון: 0.5 גרם, שומן: 0.1 גרם, פחמימות: 15 גרם', true, false, false, 710),  
('מיץ רימונים טבעי', 'רימון', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מיץ רימונים'), 12.00, 'liter', 'אנרגיה: 54 קלוריות, חלבון: 0.2 גרם, שומן: 0.3 גרם, פחמימות: 13 גרם', true, false, false, 490),  
('שום', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום'), 3.00, 'kg', 'אנרגיה: 149 קלוריות, חלבון: 6.4 גרם, שומן: 0.5 גרם, פחמימות: 33 גרם', false, false, false, 870),  
('שום כתוש', 'תבליני הארץ', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום כתוש'), 5.00, 'item', 'אנרגיה: 150 קלוריות, חלבון: 6 גרם, שומן: 0.5 גרם, פחמימות: 34 גרם', false, false, false, 730),  
('שום שחור', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום שחור'), 20.00, 'kg', 'אנרגיה: 180 קלוריות, חלבון: 7 גרם, שומן: 0.6 גרם, פחמימות: 40 גרם', false, false, false, 520),  
('תפוח עץ ירוק', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח עץ ירוק'), 8.00, 'kg', 'אנרגיה: 52 קלוריות, חלבון: 0.3 גרם, שומן: 0.2 גרם, פחמימות: 14 גרם', false, false, false, 820),  
('תפוח עץ אדום', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח עץ אדום'), 7.50, 'kg', 'אנרגיה: 53 קלוריות, חלבון: 0.3 גרם, שומן: 0.2 גרם, פחמימות: 14 גרם', false, false, false, 970),  
('תפוח עץ זהוב', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח עץ זהוב'), 8.50, 'kg', 'אנרגיה: 55 קלוריות, חלבון: 0.3 גרם, שומן: 0.2 גרם, פחמימות: 15 גרם', false, false, false, 690),  
('תפוח עץ פינק ליידי', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח עץ פינק ליידי'), 9.00, 'kg', 'אנרגיה: 54 קלוריות, חלבון: 0.3 גרם, שומן: 0.2 גרם, פחמימות: 14 גרם', false, false, false, 450),  
('בננה טרופית', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בננה'), 6.00, 'kg', 'אנרגיה: 89 קלוריות, חלבון: 1.1 גרם, שומן: 0.3 גרם, פחמימות: 23 גרם', true, false, false, 850),  
('בננה בייבי', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בננה בייבי'), 7.00, 'kg', 'אנרגיה: 90 קלוריות, חלבון: 1 גרם, שומן: 0.3 גרם, פחמימות: 23 גרם', true, false, false, 710),  
('בננה אורגנית', 'חקלאי אורגני', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בננה'), 8.00, 'kg', 'אנרגיה: 88 קלוריות, חלבון: 1 גרם, שומן: 0.3 גרם, פחמימות: 22 גרם', true, false, false, 490),  
('אבוקדו טבעי', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'אבוקדו'), 12.00, 'item', 'אנרגיה: 160 קלוריות, חלבון: 2 גרם, שומן: 15 גרם, פחמימות: 9 גרם', false, false, true, 920),  
('אבוקדו האס', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'אבוקדו'), 13.00, 'item', 'אנרגיה: 167 קלוריות, חלבון: 2 גרם, שומן: 16 גרם, פחמימות: 8.6 גרם', false, false, true, 680),  
('אבוקדו ריד', 'חקלאי ישראלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'אבוקדו'), 11.50, 'item', 'אנרגיה: 160 קלוריות, חלבון: 2 גרם, שומן: 15 גרם, פחמימות: 9 גרם', false, false, true, 470),  
('לחם לבן טרי', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'לחם לבן'), 5.00, 'item', 'אנרגיה: 266 קלוריות, חלבון: 7.9 גרם, שומן: 3.3 גרם, פחמימות: 49 גרם', true, false, false, 1500),  
('לחם מלא', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'לחם מלא'), 7.00, 'item', 'אנרגיה: 247 קלוריות, חלבון: 8.8 גרם, שומן: 4.3 גרם, פחמימות: 41 גרם', true, false, false, 1100),  
('לחם שיפון', 'מאפיית ברמן', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'לחם שיפון'), 6.50, 'item', 'אנרגיה: 259 קלוריות, חלבון: 8 גרם, שומן: 3 גרם, פחמימות: 50 גרם', true, false, false, 820),  
('לחם דגנים', 'מאפיית אנגל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'לחם דגנים'), 8.00, 'item', 'אנרגיה: 245 קלוריות, חלבון: 9 גרם, שומן: 4 גרם, פחמימות: 40 גרם', true, false, false, 580),  
('חמאה תנובה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חמאה'), 12.00, 'item', 'אנרגיה: 717 קלוריות, חלבון: 0.9 גרם, שומן: 81 גרם, פחמימות: 0.1 גרם', false, false, true, 1850),  
('חמאה מלוחה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חמאה מלוחה'), 12.50, 'item', 'אנרגיה: 717 קלוריות, חלבון: 0.9 גרם, שומן: 81 גרם, פחמימות: 0.1 גרם', false, true, true, 920),  
('תחליף חמאה מרגרינה', 'בלו בנד', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מרגרינה'), 8.00, 'item', 'אנרגיה: 720 קלוריות, חלבון: 0 גרם, שומן: 80 גרם, פחמימות: 0 גרם', false, false, true, 1000),  
('גבינת צהובה תנובה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת צהובה'), 16.00, 'item', 'אנרגיה: 402 קלוריות, חלבון: 23.3 גרם, שומן: 32 גרם, פחמימות: 1 גרם', false, false, true, 970),  
('גבינת פטה תנובה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת פטה'), 14.00, 'item', 'אנרגיה: 264 קלוריות, חלבון: 14 גרם, שומן: 21 גרם, פחמימות: 4 גרם', false, false, true, 680),  
('גבינת קממבר', 'גד', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת קממבר'), 18.00, 'item', 'אנרגיה: 300 קלוריות, חלבון: 19 גרם, שומן: 24 גרם, פחמימות: 0 גרם', false, false, true, 520),  
('גבינת קוטג 5 אחוז', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת קוטג'), 8.50, 'item', 'אנרגיה: 101 קלוריות, חלבון: 11.2 גרם, שומן: 5 גרם, פחמימות: 2.8 גרם', false, false, false, 1800),  
('גבינת קוטג 9 אחוז', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת קוטג'), 9.00, 'item', 'אנרגיה: 140 קלוריות, חלבון: 11 גרם, שומן: 9 גרם, פחמימות: 3 גרם', false, false, true, 920),  
('גבינת קוטג דלת שומן 1%', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינת קוטג'), 8.00, 'item', 'אנרגיה: 72 קלוריות, חלבון: 11 גרם, שומן: 1 גרם, פחמימות: 3 גרם', false, false, false, 570),  
('דבש טהור', 'דבשיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'דבש'), 18.00, 'kg', 'אנרגיה: 304 קלוריות, חלבון: 0.3 גרם, שומן: 0 גרם, פחמימות: 82 גרם', true, false, false, 910),  
('דבש פרחי בר', 'דבשית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'דבש'), 20.00, 'kg', 'אנרגיה: 310 קלוריות, חלבון: 0.5 גרם, שומן: 0 גרם, פחמימות: 80 גרם', true, false, false, 680),  
('סילאן טבעי', 'תמרים', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סילאן'), 15.00, 'kg', 'אנרגיה: 280 קלוריות, חלבון: 1 גרם, שומן: 0 גרם, פחמימות: 75 גרם', true, false, false, 520),  
('שוקולד פררו רושה', 'פררו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד פררו רושה'), 20.00, 'item', 'אנרגיה: 604 קלוריות, חלבון: 7.5 גרם, שומן: 37 גרם, פחמימות: 51 גרם', true, false, true, 940),  
('שוקולד לינדט 85%', 'לינדט', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד מריר'), 13.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 9 גרם, שומן: 42 גרם, פחמימות: 19 גרם', true, false, true, 720),  
('שוקולד מילקה אגוזים', 'מילקה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב'), 9.00, 'item', 'אנרגיה: 530 קלוריות, חלבון: 8 גרם, שומן: 29 גרם, פחמימות: 61 גרם', true, false, true, 470),  
('חטיף ציפס טורטיה', 'קונציטו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ציפס'), 5.00, 'item', 'אנרגיה: 490 קלוריות, חלבון: 6 גרם, שומן: 30 גרם, פחמימות: 60 גרם', true, true, false, 800),  
('חטיף במבה', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'במבה'), 4.00, 'item', 'אנרגיה: 544 קלוריות, חלבון: 16 גרם, שומן: 34 גרם, פחמימות: 46 גרם', true, true, false, 1900),  
('חטיף ביסלי גריל', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביסלי'), 4.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 8 גרם, שומן: 25 גרם, פחמימות: 62 גרם', true, true, false, 650),  
('חטיף פריכיות אורז', 'אורזיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פריכיות'), 10.00, 'item', 'אנרגיה: 391 קלוריות, חלבון: 7 גרם, שומן: 2 גרם, פחמימות: 83 גרם', false, false, false, 920),  
('פריכיות תירס', 'תירסיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פריכיות תירס'), 9.00, 'item', 'אנרגיה: 380 קלוריות, חלבון: 8 גרם, שומן: 3 גרם, פחמימות: 80 גרם', false, false, false, 710),  
('פריכיות כוסמין', 'כוסמיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פריכיות כוסמין'), 11.00, 'item', 'אנרגיה: 370 קלוריות, חלבון: 9 גרם, שומן: 2 גרם, פחמימות: 78 גרם', false, false, false, 470),  
('קולה', 'קוקה קולה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קולה'), 6.00, 'liter', 'אנרגיה: 42 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 11 גרם', true, false, false, 950),  
('קולה זירו', 'קוקה קולה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קולה'), 6.50, 'liter', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 680),  
('ספרייט', 'קוקה קולה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ספרייט'), 6.00, 'liter', 'אנרגיה: 38 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 9.6 גרם', true, false, false, 520),  
('פיצוחים תפזורת', 'פיצוחי הארץ', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פיצוחים'), 15.00, 'kg', 'אנרגיה: 600 קלוריות, חלבון: 20 גרם, שומן: 50 גרם, פחמימות: 20 גרם', true, true, true, 930),  
('גרעיני חמנייה קלויים', 'פיצוחי הארץ', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גרעיני חמנייה'), 12.00, 'kg', 'אנרגיה: 584 קלוריות, חלבון: 20 גרם, שומן: 51 גרם, פחמימות: 20 גרם', true, true, true, 710),  
('פיסטוקים קלויים', 'פיצוחי הארץ', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פיסטוקים'), 25.00, 'kg', 'אנרגיה: 562 קלוריות, חלבון: 21 גרם, שומן: 45 גרם, פחמימות: 28 גרם', true, true, true, 490),  
('חטיף אגוזים ושוקולד', 'אבן גבירול', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף אגוזים'), 8.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 10 גרם, שומן: 30 גרם, פחמימות: 40 גרם', true, false, true, 890),  
('חטיף גרנולה ודבש', 'נייצ''ר ואלי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף גרנולה'), 6.00, 'item', 'אנרגיה: 400 קלוריות, חלבון: 6 גרם, שומן: 14 גרם, פחמימות: 68 גרם', true, false, false, 730),  
('חטיף פירות יבשים', 'בריאות פלוס', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף פירות'), 7.00, 'item', 'אנרגיה: 350 קלוריות, חלבון: 3 גרם, שומן: 1 גרם, פחמימות: 85 גרם', true, false, false, 520),  
('קרמבו', 'מאפיית שקד', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קרמבו'), 2.50, 'item', 'אנרגיה: 80 קלוריות, חלבון: 1 גרם, שומן: 3.5 גרם, פחמימות: 12 גרם', true, false, false, 970),  
('מרשמלו', 'סוכריה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מרשמלו'), 5.00, 'item', 'אנרגיה: 318 קלוריות, חלבון: 2 גרם, שומן: 0 גרם, פחמימות: 81 גרם', true, false, false, 680),  
('סוכריות גומי', 'גומיה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכריות גומי'), 4.00, 'item', 'אנרגיה: 350 קלוריות, חלבון: 6 גרם, שומן: 0 גרם, פחמימות: 80 גרם', true, false, false, 440),  
('סודה', 'הכחול לבן', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סודה'), 4.00, 'liter', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 890),  
('מים מינרליים', 'מי עדן', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מים'), 3.00, 'liter', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 710),  
('מים מוגזים', 'סודה סטרים', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מים מוגזים'), 5.00, 'liter', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 430),  
('חטיף תפו"א', 'תפוציפס', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף תפו"א'), 5.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 6 גרם, שומן: 30 גרם, פחמימות: 60 גרם', true, true, true, 980),  
('חטיף תפו"א אפוי', 'תפוציפס', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף תפו"א'), 5.50, 'item', 'אנרגיה: 450 קלוריות, חלבון: 7 גרם, שומן: 20 גרם, פחמימות: 70 גרם', true, true, false, 670),  
('חטיף תפו"א מופחת מלח', 'תפוציפס', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף תפו"א'), 5.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 6 גרם, שומן: 30 גרם, פחמימות: 60 גרם', true, false, true, 510),  
('מאפה זיתים', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מאפה זיתים'), 10.00, 'item', 'אנרגיה: 250 קלוריות, חלבון: 5 גרם, שומן: 15 גרם, פחמימות: 30 גרם', false, false, true, 710),  
('מאפה גבינה', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'מאפה גבינה'), 11.00, 'item', 'אנרגיה: 300 קלוריות, חלבון: 8 גרם, שומן: 18 גרם, פחמימות: 32 גרם', false, false, true, 970),  
('בורקס תפוח אדמה', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בורקס'), 9.00, 'item', 'אנרגיה: 350 קלוריות, חלבון: 6 גרם, שומן: 20 גרם, פחמימות: 35 גרם', false, false, true, 460),  
('סלט חצילים', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סלט חצילים'), 8.00, 'kg', 'אנרגיה: 90 קלוריות, חלבון: 1 גרם, שומן: 5 גרם, פחמימות: 12 גרם', false, false, false, 540),  
('סלט טחינה', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'טחינה'), 10.00, 'kg', 'אנרגיה: 595 קלוריות, חלבון: 17 גרם, שומן: 54 גרם, פחמימות: 21 גרם', false, false, true, 810),  
('סלט חומוס', 'שוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חומוס'), 9.00, 'kg', 'אנרגיה: 166 קלוריות, חלבון: 8 גרם, שומן: 9 גרם, פחמימות: 14 גרם', false, false, false, 950),  
('רוטב פסטו', 'מאפיית ישראל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב פסטו'), 15.00, 'item', 'אנרגיה: 600 קלוריות, חלבון: 10 גרם, שומן: 60 גרם, פחמימות: 4 גרם', false, true, true, 920),  
('רוטב אלפרדו', 'פרגו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב אלפרדו'), 13.00, 'item', 'אנרגיה: 150 קלוריות, חלבון: 3 גרם, שומן: 14 גרם, פחמימות: 3 גרם', false, true, true, 730),  
('רוטב סויה', 'קיקומן', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב סויה'), 7.00, 'item', 'אנרגיה: 53 קלוריות, חלבון: 8 גרם, שומן: 0 גרם, פחמימות: 4 גרם', false, true, false, 520),  
('חלב סויה תנובה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב סויה'), 8.00, 'liter', 'אנרגיה: 54 קלוריות, חלבון: 3 גרם, שומן: 2 גרם, פחמימות: 6 גרם', false, false, false, 720),  
('חלב שקדים אלפרו', 'אלפרו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב שקדים'), 9.50, 'liter', 'אנרגיה: 24 קלוריות, חלבון: 0.5 גרם, שומן: 1.1 גרם, פחמימות: 3 גרם', false, false, false, 610),  
('חלב שיבולת שועל Oatly', 'Oatly', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב שיבולת שועל'), 10.00, 'liter', 'אנרגיה: 46 קלוריות, חלבון: 1 גרם, שומן: 1.5 גרם, פחמימות: 6.7 גרם', false, false, false, 580),  
('גבינה טבעונית Violife', 'Violife', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גבינה טבעונית'), 15.00, 'item', 'אנרגיה: 270 קלוריות, חלבון: 0 גרם, שומן: 21 גרם, פחמימות: 20 גרם', false, false, true, 540),  
('יוגורט ללא לקטוז Yoplait', 'Yoplait', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'יוגורט ללא לקטוז'), 6.00, 'item', 'אנרגיה: 60 קלוריות, חלבון: 5 גרם, שומן: 2 גרם, פחמימות: 5 גרם', false, false, false, 640),  
('חלב ללא לקטוז תנובה', 'תנובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב ללא לקטוז'), 7.50, 'liter', 'אנרגיה: 50 קלוריות, חלבון: 3.3 גרם, שומן: 1 גרם, פחמימות: 5 גרם', false, false, false, 590),  
('לחם ללא גלוטן בר-אל', 'בר-אל', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'לחם ללא גלוטן'), 12.00, 'item', 'אנרגיה: 220 קלוריות, חלבון: 2 גרם, שומן: 3 גרם, פחמימות: 44 גרם', true, false, false, 840),  
('פסטה ללא גלוטן ברילה', 'ברילה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה ללא גלוטן'), 9.00, 'kg', 'אנרגיה: 350 קלוריות, חלבון: 7 גרם, שומן: 1 גרם, פחמימות: 75 גרם', true, false, false, 650),  
('קמח ללא גלוטן אסם', 'אסם', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח ללא גלוטן'), 10.00, 'kg', 'אנרגיה: 350 קלוריות, חלבון: 2 גרם, שומן: 1 גרם, פחמימות: 85 גרם', true, false, false, 580),  
('גרנולה ללא גלוטן גרנוליה', 'גרנוליה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גרנולה ללא גלוטן'), 18.00, 'kg', 'אנרגיה: 400 קלוריות, חלבון: 10 גרם, שומן: 15 גרם, פחמימות: 60 גרם', true, false, false, 890),  
('אטריות אורז אורגני', 'אורגני', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'אטריות אורז'), 7.00, 'kg', 'אנרגיה: 350 קלוריות, חלבון: 6 גרם, שומן: 1 גרם, פחמימות: 80 גרם', true, false, false, 480),  
('שוקולד ללא סוכר עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד ללא סוכר'), 10.00, 'item', 'אנרגיה: 460 קלוריות, חלבון: 5 גרם, שומן: 35 גרם, פחמימות: 60 גרם (מתוכם סוכרים 0 גרם)', false, false, true, 430),  
('סוכרלוז סוכרזית', 'סוכרזית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכרלוז'), 12.00, 'item', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 570),  
('סטיביה מתוק', 'סטיביה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סטיביה'), 15.00, 'item', 'אנרגיה: 0 קלוריות, חלבון: 0 גרם, שומן: 0 גרם, פחמימות: 0 גרם', false, false, false, 620),  
('ממרח שוקולד ללא אגוזים השחר', 'השחר', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ממרח שוקולד ללא אגוזים'), 8.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 4 גרם, שומן: 25 גרם, פחמימות: 65 גרם', true, false, false, 950),  
('חטיף חלבון טבעוני פרו', 'פרו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חטיף חלבון טבעוני'), 12.00, 'item', 'אנרגיה: 200 קלוריות, חלבון: 20 גרם, שומן: 6 גרם, פחמימות: 15 גרם', false, false, false, 660),  
('עוגיות טבעוניות שוקולד ציפס', 'עוגיה טובה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עוגיות טבעוניות'), 15.00, 'item', 'אנרגיה: 450 קלוריות, חלבון: 5 גרם, שומן: 20 גרם, פחמימות: 60 גרם', true, false, true, 520),  
('טופו סאנקי', 'סאנקי', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'טופו'), 8.00, 'item', 'אנרגיה: 76 קלוריות, חלבון: 8 גרם, שומן: 4.8 גרם, פחמימות: 1.9 גרם', false, false, false, 540),  
('נקניקיות טבעוניות טבעול', 'טבעול', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'נקניקיות טבעוניות'), 15.00, 'item', 'אנרגיה: 180 קלוריות, חלבון: 14 גרם, שומן: 7 גרם, פחמימות: 14 גרם', false, false, false, 620),  
('שוקולד טבעוני עלית', 'עלית', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד טבעוני'), 9.00, 'item', 'אנרגיה: 500 קלוריות, חלבון: 5 גרם, שומן: 30 גרם, פחמימות: 60 גרם', true, false, true, 370),  
('גלידה טבעונית אלפרו', 'אלפרו', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גלידה טבעונית'), 20.00, 'item', 'אנרגיה: 180 קלוריות, חלבון: 2 גרם, שומן: 9 גרם, פחמימות: 23 גרם', true, false, true, 490),  
('חלב קוקוס חלביה', 'חלביה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב קוקוס'), 7.00, 'liter', 'אנרגיה: 230 קלוריות, חלבון: 2.3 גרם, שומן: 24 גרם, פחמימות: 6 גרם', false, false, true, 290);  
"""  
cursor.execute(insert_groceries)  

140

In [26]:
insert_bridge_table_groceries_to_additional_health_tags = """
INSERT INTO bridge_table_groceries_to_additional_health_tags (barcode, health_tag_id)
VALUES
-- Milk and Dairy Products

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב תנובה 3 אחוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב 1 אחוז טרה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב יטבתה 3 אחוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב הרדוף 3 אחוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Yogurts
((SELECT barcode FROM groceries WHERE grocery_name = 'יוגורט דנונה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'יוגורט פרו ביוטי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- 'יוגורט יווני 10%' with two health tags
((SELECT barcode FROM groceries WHERE grocery_name = 'יוגורט יווני 10%'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'יוגורט יווני 10%'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'עלול להכיל אגוזים')),

-- Cheeses
((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת צהובה תנובה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת פטה תנובה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת קממבר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת קוטג 5 אחוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת קוטג 9 אחוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינת קוטג דלת שומן 1%'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
-- Lactose-Free Dairy Alternatives
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב סויה תנובה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב סויה תנובה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב שקדים אלפרו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב שקדים אלפרו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב שיבולת שועל Oatly'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב שיבולת שועל Oatly'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גבינה טבעונית Violife'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גבינה טבעונית Violife'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'יוגורט ללא לקטוז Yoplait'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב ללא לקטוז תנובה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

-- Eggs
((SELECT barcode FROM groceries WHERE grocery_name = 'ביצים טריות צבר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'ביצי חופש'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'ביצי אומגה 3'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Oils
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית יד מרדכי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית יד מרדכי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית יד מרדכי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית כתית מעולה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית כתית מעולה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן זית כתית מעולה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'שמן קנולה קנוליה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שמן קנולה קנוליה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Salt
((SELECT barcode FROM groceries WHERE grocery_name = 'מלח דק סול'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'מלח דק סול'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מלח גס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'מלח גס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מלח הימלאיה ורוד'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'מלח הימלאיה ורוד'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Sugar
((SELECT barcode FROM groceries WHERE grocery_name = 'סוכר לבן נוגה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סוכר לבן סוגת'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סוכר חום דמררה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Coffee and Tea
((SELECT barcode FROM groceries WHERE grocery_name = 'קפה טורקי ארקפה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'קפה נמס עלית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'קפה פילטר סטארבקס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תה ירוק עלית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תה שחור ויסוצקי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תה צמחים קמומיל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Fruits and Vegetables

-- Potatoes
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה לבן רמי לוי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה לבן רמי לוי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה לבן רמי לוי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה אדום שופרסל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה אדום שופרסל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה אדום שופרסל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה בייבי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה בייבי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה בייבי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

-- Carrots
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר אורגני טיב טעם'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר אורגני טיב טעם'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר אורגני טיב טעם'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גזר טרי חקלאי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר טרי חקלאי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר טרי חקלאי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גזר בייבי ארוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'גזר בייבי ארוז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Tomatoes
((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה שרי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה שרי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה תמר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'עגבנייה תמר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Onions
((SELECT barcode FROM groceries WHERE grocery_name = 'בצל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בצל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בצל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),


((SELECT barcode FROM groceries WHERE grocery_name = 'בצל ירוק'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בצל ירוק'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'בצל סגול'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בצל סגול'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Garlic
((SELECT barcode FROM groceries WHERE grocery_name = 'שום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'שום כתוש'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שום כתוש'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'שום שחור'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שום שחור'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Apples
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ ירוק'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ ירוק'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ אדום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ אדום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ זהוב'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ זהוב'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ פינק ליידי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ פינק ליידי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Bananas
((SELECT barcode FROM groceries WHERE grocery_name = 'בננה טרופית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בננה טרופית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'בננה בייבי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בננה בייבי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'בננה אורגנית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'בננה אורגנית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Avocados
((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'מאושר ע"י האגודה הישראלית לסוכרת')),

((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו האס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו האס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו ריד'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'אבוקדו ריד'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Sauces
((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות רוטב אדום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות רוטב אדום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות בזיליקום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות בזיליקום'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות פיקנטי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב עגבניות פיקנטי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב פסטו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'עלול להכיל אגוזים')),

((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב אלפרדו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'עלול להכיל אגוזים')),

((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב סויה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'רוטב סויה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Drinks
((SELECT barcode FROM groceries WHERE grocery_name = 'מיץ תפוזים פרימור'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מיץ ענבים טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מיץ רימונים טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'קולה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'קולה זירו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'ספרייט'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סודה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מים מינרליים'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'מים מוגזים'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Honey and Spreads
((SELECT barcode FROM groceries WHERE grocery_name = 'דבש טהור'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'דבש פרחי בר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סילאן טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),
((SELECT barcode FROM groceries WHERE grocery_name = 'סילאן טבעי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

((SELECT barcode FROM groceries WHERE grocery_name = 'ממרח שוקולד ללא אגוזים השחר'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),
 -- Snacks
((SELECT barcode FROM groceries WHERE grocery_name = 'חטיף ציפס טורטיה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חטיף פריכיות אורז'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'פריכיות תירס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חטיף חלבון טבעוני פרו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חטיף פירות יבשים'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

-- Gluten-Free Products
((SELECT barcode FROM groceries WHERE grocery_name = 'לחם ללא גלוטן בר-אל'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'פסטה ללא גלוטן ברילה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'קמח ללא גלוטן אסם'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גרנולה ללא גלוטן גרנוליה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'אטריות אורז אורגני'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Sugar-Free Products
((SELECT barcode FROM groceries WHERE grocery_name = 'שוקולד ללא סוכר עלית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סוכרלוז סוכרזית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

((SELECT barcode FROM groceries WHERE grocery_name = 'סטיביה מתוק'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'ללא גלוטן')),

-- Vegan Products
((SELECT barcode FROM groceries WHERE grocery_name = 'טופו סאנקי'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'נקניקיות טבעוניות טבעול'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'שוקולד טבעוני עלית'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'גלידה טבעונית אלפרו'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),

((SELECT barcode FROM groceries WHERE grocery_name = 'חלב קוקוס חלביה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב קוקוס חלביה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),

-- Products That May Contain Nuts
((SELECT barcode FROM groceries WHERE grocery_name = 'חטיף אגוזים ושוקולד'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'עלול להכיל אגוזים')),

((SELECT barcode FROM groceries WHERE grocery_name = 'עוגיות טבעוניות שוקולד ציפס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני')),
((SELECT barcode FROM groceries WHERE grocery_name = 'עוגיות טבעוניות שוקולד ציפס'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'עלול להכיל אגוזים')),

-- Margarine
((SELECT barcode FROM groceries WHERE grocery_name = 'תחליף חמאה מרגרינה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'לא מכיל לקטוז')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תחליף חמאה מרגרינה'),
 (SELECT health_tag_id FROM additional_health_tags WHERE health_tag_name = 'טבעוני'));


"""

cursor.execute(insert_bridge_table_groceries_to_additional_health_tags)


157

In [27]:

# Insert data into recipes
insert_recipes = """
INSERT INTO recipes (recipe_name, instructions_and_ingredients, embedding_vector, recipe_category)
VALUES
('פאי תפוחים עם קרום פריך', 'מצרכים: תפוחים (4 יחידות), קמח (200 גרם), סוכר (100 גרם). הוראות הכנה: 1. מכינים את הבצק ומניחים בתבנית. 2. מכינים את מילוי התפוחים ומניחים מעל הבצק. 3. אופים בחום בינוני עד שהפאי זהוב ופריך.', NULL, 'קינוחים'),
('מוס שוקולד מריר', 'מצרכים: שוקולד מריר (200 גרם), שמנת מתוקה (200 מ"ל), סוכר (50 גרם). הוראות הכנה: 1. ממיסים את השוקולד ושמים בצד. 2. מקציפים את השמנת עם הסוכר. 3. מערבבים את השוקולד עם השמנת המוקצפת. 4. מקררים לפני ההגשה.', NULL, 'קינוחים'),
('סלט ירקות עם גבינה בולגרית', 'מצרכים: עגבניות (3 יחידות), מלפפונים (2 יחידות), גבינה בולגרית (100 גרם), שמן זית (2 כפות). הוראות הכנה: 1. חותכים את הירקות לחתיכות קטנות. 2. מוסיפים את הגבינה ואת שמן הזית. 3. מערבבים היטב ומגישים.', NULL, 'סלטים'),
('פסטה ברוטב עגבניות', 'מצרכים: פסטה (250 גרם), רוטב עגבניות (1 כוס), בצל (1 יחידה), שום (2 שיניים). הוראות הכנה: 1. מבשלים את הפסטה לפי ההוראות על האריזת. 2. מכינים את הרוטב על ידי טיגון הבצל והשום, הוספת רוטב העגבניות ובישול עוד כ-10 דקות. 3. מערבבים את הפסטה עם הרוטב ומגישים.', NULL, 'מנה עיקרית'),
('פנקייקים קלאסיים', 'מצרכים: קמח (200 גרם), חלב (300 מ"ל), ביצים (2 יחידות), סוכר (2 כפות). הוראות הכנה: מערבבים את כל החומרים ומטגנים במחבת חמה עד להזהבה.', NULL, 'ארוחות בוקר'),
('מרק ירקות בריא', 'מצרכים: תפוח אדמה (2 יחידות), גזר (2 יחידות), בצל (1 יחידה), שום (2 שיניים), שמן זית (2 כפות), מלח (כפית), מים (ליטר). הוראות הכנה: 1. מטגנים את הבצל והשום. 2. מוסיפים את הירקות ומטגנים כ-5 דקות. 3. מוסיפים מים, מלח ומבשלים עד שהירקות רכים.', NULL, 'מרקים'),
('שקשוקה עם ביצים', 'מצרכים: ביצים (4 יחידות), עגבניות (3 יחידות), בצל (1 יחידה), פלפל חריף (1 יחידה), שמן זית (2 כפות). הוראות הכנה: מטגנים את הירקות, מוסיפים את הביצים ומבשלים עד שהן מוכנות.', NULL, 'ארוחות בוקר'),
('עוגת שוקולד קלה', 'מצרכים: שוקולד מריר (200 גרם), חמאה (100 גרם), קמח (150 גרם), סוכר (100 גרם), ביצים (3 יחידות). הוראות הכנה: ממיסים את השוקולד והחמאה. מערבבים עם שאר החומרים ואופים ב-180 מעלות למשך 25 דקות.', NULL, 'קינוחים');
"""

cursor.execute(insert_recipes)

8

In [28]:

# Insert data into discounts
insert_discounts = """
INSERT INTO discounts (discount_name, grocery_type_id, discount_type, discount_value, grocery_category, grocery_base_category)
VALUES
('הנחה על חלב במחיר מיוחד של שקל', 
 (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז'), 
 'fixed_price', 
 1.00,
 (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז'),
 (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז')
),
('הנחה של 10% על שוקולד חלב - מתוק יותר', 
 (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב'), 
 'percentage', 
 10.00,
 (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב'),
 (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'שוקולד חלב')
),
('15% הנחה על תפוחי אדמה לבן - טעים וטרי', 
 (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה לבן'), 
 'percentage', 
 15.00,
 (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה לבן'),
 (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה לבן')
),
('רק 2 ₪ על קפה - לפתוח את היום ברגל ימין', 
 (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קפה'), 
 'fixed_price', 
 2.00,
 (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'קפה'),
 (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'קפה')
),
('מבצע 50% הנחה על תפוח עץ ירוק', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח עץ ירוק'), 'percentage', 0.50, (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'תפוח עץ ירוק'), (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'תפוח עץ ירוק')),
('הנחה על פסטה', (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה'), 'fixed_price', 4.00, (SELECT grocery_category FROM groceries_types WHERE grocery_type_name = 'פסטה'), (SELECT grocery_base_category FROM groceries_types WHERE grocery_type_name = 'פסטה'));
"""

cursor.execute(insert_discounts)

6

In [29]:



# Insert data into bridge_table_products_types_to_recipes
# Insert data into bridge_table_products_types_to_recipes
insert_bridge_table_products_types_to_recipes = """
INSERT INTO bridge_table_products_types_to_recipes (recipe_id, grocery_type_id)
VALUES
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פאי תפוחים עם קרום פריך'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'תפוח אדמה לבן')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פאי תפוחים עם קרום פריך'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פאי תפוחים עם קרום פריך'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'סוכר לבן')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מוס שוקולד מריר'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שוקולד מריר')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מוס שוקולד מריר'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שמן זית')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'סלט ירקות עם גבינה בולגרית'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'סלט ירקות עם גבינה בולגרית'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פסטה ברוטב עגבניות'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'פסטה')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פסטה ברוטב עגבניות'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'רוטב עגבניות')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פסטה ברוטב עגבניות'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בצל')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פסטה ברוטב עגבניות'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פנקייקים קלאסיים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'קמח')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פנקייקים קלאסיים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'פנקייקים קלאסיים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'חלב 3 אחוז')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מרק ירקות בריא'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מרק ירקות בריא'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'גזר')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מרק ירקות בריא'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'בצל')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'מרק ירקות בריא'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'שקשוקה עם ביצים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'עגבנייה')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'שקשוקה עם ביצים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'ביצים')),
((SELECT recipe_id FROM recipes WHERE recipe_name = 'שקשוקה עם ביצים'), (SELECT grocery_type_id FROM groceries_types WHERE grocery_type_name = 'שום'));
"""

cursor.execute(insert_bridge_table_products_types_to_recipes)


21

In [30]:

# Insert data into bridge_table_discounts_to_groceries
insert_bridge_table_discounts_to_groceries = """
INSERT INTO bridge_table_discounts_to_groceries (barcode, discount_id)
VALUES
((SELECT barcode FROM groceries WHERE grocery_name = 'חלב תנובה 3 אחוז'), (SELECT discount_id FROM discounts WHERE discount_name = 'הנחה על חלב במחיר מיוחד של שקל')),
((SELECT barcode FROM groceries WHERE grocery_name = 'שוקולד חלב עלית'), (SELECT discount_id FROM discounts WHERE discount_name = 'הנחה של 10% על שוקולד חלב - מתוק יותר')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח אדמה לבן רמי לוי'), (SELECT discount_id FROM discounts WHERE discount_name = '15% הנחה על תפוחי אדמה לבן - טעים וטרי')),
((SELECT barcode FROM groceries WHERE grocery_name = 'קפה טורקי ארקפה'), (SELECT discount_id FROM discounts WHERE discount_name = 'רק 2 ₪ על קפה - לפתוח את היום ברגל ימין')),
((SELECT barcode FROM groceries WHERE grocery_name = 'תפוח עץ ירוק'), (SELECT discount_id FROM discounts WHERE discount_name = 'מבצע 50% הנחה על תפוח עץ ירוק')),
((SELECT barcode FROM groceries WHERE grocery_name = 'פסטה דל'), (SELECT discount_id FROM discounts WHERE discount_name = 'הנחה על פסטה'));
"""

cursor.execute(insert_bridge_table_discounts_to_groceries)

6

In [31]:

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

: 